# 第10章: 事前学習済み言語モデル（GPT型）

本章では、GPT型（Transformerのデコーダ型）の事前学習済みモデルを利用して、言語生成、評判分析器（ポジネガ分類器）の構築、ファインチューニング、強化学習などに取り組む。

In [18]:
%%capture
!pip install transformers, datasets
!pip install trl

## 90. 次単語予測

“The movie was full of"に続くトークン（トークン列ではなく一つのトークンであることに注意せよ）として適切なもの上位10個と、その確率（尤度）を求めよ。ただし、言語モデルへのプロンプトがどのようなトークン列に変換されたか、確認せよ。

In [15]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments
from torch.utils.data import Dataset, DataLoader
from trl import DPOTrainer #99.DPOに使用

In [3]:
%%capture
!wget https://dl.fbaipublicfiles.com/glue/data/SST-2.zip
!unzip SST-2.zip

In [ ]:
%%capture
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [ ]:
model.eval()

prompt = "The movie was full of"


encoded_input = tokenizer(prompt, return_tensors='pt')
print(f"inputs_ids shape:{encoded_input['input_ids'].shape}\nattention_mask shape:{encoded_input['attention_mask'].shape}\n")
print(encoded_input)
token = tokenizer.decode(encoded_input['input_ids'][0])
print(f"tokens: {token}\n")

with torch.no_grad():
    output = model(**encoded_input)


logits = output.logits
print(logits.shape) #shape: [B, L, V]

last_token_logits = logits[0, -1, :]  #これで一番最後のトークンの次のロジット
print(last_token_logits.shape)

last_token_logits_prob = F.softmax(last_token_logits, dim=0)

topk = torch.topk(last_token_logits_prob, k=10)

for idx, score in zip(topk.indices, topk.values):
    print(f"{tokenizer.decode(idx)}: {score.item():.4f}")

inputs_ids shape:torch.Size([1, 5])
attention_mask shape:torch.Size([1, 5])

{'input_ids': tensor([[ 464, 3807,  373, 1336,  286]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}
tokens: The movie was full of

torch.Size([1, 5, 50257])
torch.Size([50257])
 jokes: 0.0219
 great: 0.0186
 laughs: 0.0115
 bad: 0.0109
 surprises: 0.0107
 references: 0.0105
 fun: 0.0100
 humor: 0.0074
 ": 0.0074
 the: 0.0067


## 91. 続きのテキストの予測

“The movie was full of"に続くテキストを複数予測せよ。このとき、デコーディングの方法や温度パラメータ（temperature）を変えながら、予測される複数のテキストの変化を観察せよ。

In [ ]:
#greedy
import time
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

model.eval()

prompt = "The movie was full of"

encoded_input = tokenizer(prompt, return_tensors='pt')

with torch.no_grad():
  start_time = time.time()
  output = model.generate(encoded_input['input_ids'], max_length=15)
  end_time = time.time()

print(f"time: {(end_time - start_time):.4f}")
print(output.shape) #shape: [B,L]
print(tokenizer.decode(output[0], skip_special_tokens=False)) #確率が最も高いトークンを１つずつ順番に選んでいく->greedy decoding デフォはこれっぽい


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


time: 0.9133
torch.Size([1, 15])
The movie was full of jokes and jokes about how the movie was a joke


In [ ]:
#beam_serch
import time
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

model.eval()

prompt = "The movie was full of"

encoded_input = tokenizer(prompt, return_tensors='pt')

with torch.no_grad():
  start_time = time.time()
  output = model.generate(encoded_input['input_ids'], max_length=15, num_beams=5)
  end_time = time.time()

print(f"time: {(end_time - start_time):.3f}s")
print(output.shape) #shape: [B,L]
print(tokenizer.decode(output[0], skip_special_tokens=False)) #ビームサーチは貪欲法と比較して処理時間がおそい．


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


time: 1.820s
torch.Size([1, 15])
The movie was full of jokes and jokes and jokes and jokes and jokes and


In [ ]:
#topk
import time
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

model.eval()

prompt = "The movie was full of"

encoded_input = tokenizer(prompt, return_tensors='pt')

with torch.no_grad():
  start_time = time.time()
  output = model.generate(
      encoded_input['input_ids'],
      max_length=15,
      top_k=50,
      top_p=0.9,
      temperature=2.0)
  end_time = time.time()

print(f"time: {(end_time - start_time):.3f}s")
print(output.shape) #shape: [B,L]
print(tokenizer.decode(output[0], skip_special_tokens=False))

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


time: 0.812s
torch.Size([1, 15])
The movie was full of jokes and jokes about how the movie was a joke


## 92. 予測されたテキストの確率を計算

“The movie was full of"に続くテキストを予測し、生成された各単語の尤度を表示せよ（生成されるテキストが長いと出力が読みにくくなるので、適当な長さで生成を打ち切るとよい）。

In [ ]:
import time

model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

prompt = "The movie was full of"
encoded_input = tokenizer(prompt, return_tensors='pt')
input_ids = encoded_input['input_ids']


with torch.no_grad():
  start_time = time.time()

  output = model.generate(
      input_ids,
      max_length=15,
      top_k=50,
      top_p=1.0,
      temperature=1.0,
      return_dict_in_generate=True,
      output_scores=True
  )
  end_time = time.time()



generated_ids = output.sequences[0]
print(f"generated_ids shape: {generated_ids.shape}")
generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
print(f"time: {(end_time - start_time):.3f}s")
print(f"\nGenerated Text:\n{generated_text}\n")

print(len(output.scores)) #tuple: max_len - input_len (=generated_len)
print(output.scores[0].shape) #shape: [1, vocab_size]

for score in output.scores:
  prob = F.softmax(score, dim=1)
  vocab_ids = torch.argmax(score, dim=1) #shape: [batch_size, ]
  tokens = tokenizer.decode(vocab_ids)
  print(f"{tokens}: P={prob[0, vocab_ids].item():.3f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


generated_ids shape: torch.Size([15])
time: 1.279s

Generated Text:
The movie was full of jokes and jokes about how the movie was a joke

10
torch.Size([1, 50257])
 jokes: P=0.022
 and: P=0.289
 jokes: P=0.099
 about: P=0.206
 how: P=0.100
 the: P=0.085
 movie: P=0.036
 was: P=0.296
 a: P=0.068
 joke: P=0.174


## 93. パープレキシティ

適当な文を準備して、事前学習済み言語モデルでパープレキシティを測定せよ。例えば、

+ The movie was full of surprises
+ The movies were full of surprises
+ The movie were full of surprises
+ The movies was full of surprises

の4文に対して、パープレキシティを測定して観察せよ（最後の2つの文は故意に文法的な間違いを入れた）。

In [ ]:
import math

model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

sentences = ['The movie was full of surprises',
             'The movies were full of surprises',
             'The movie were full of surprises',
             'The movies was full of surprises']

def cal_ppl(sentence):
    inputs = tokenizer(sentence, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs['input_ids']) #loss = -1/L Sigma log(wt|wt<)
        loss = outputs.loss
        ppl = math.exp(loss.item()) #tensorだったからitem()でfloat型に変換
    return ppl

def my_cal_ppl(sentence):
    inputs = tokenizer(sentence, return_tensors='pt')
    ids = inputs['input_ids'][0]  # shape: [seq_len]

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits  # shape: [1, seq_len, vocab_size]
    log_probs = F.log_softmax(logits, dim=2)  # softmaxしてlogとる

    loss_sum = 0.0
    token_num = len(ids) - 1

    # t番目の出力は t+1 番目のトークンを予測している
    for i in range(1, len(ids)):
        true_token_id = ids[i].item()
        log_prob = log_probs[0, i - 1, true_token_id].item()
        loss_sum += log_prob

    ppl = math.exp(-loss_sum / token_num)
    return ppl

ppl = [cal_ppl(sentence) for sentence in sentences]
print(ppl)

my_ppl = [my_cal_ppl(sentence) for sentence in sentences]
print(my_ppl)






`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


[99.35493255487346, 126.480708931546, 278.879620238215, 274.6586517692294]
[99.35488614108807, 126.4806989425864, 278.8796867283058, 274.65871848078285]


## 94. チャットテンプレート

"What do you call a sweet eaten after dinner?"という問いかけに対する応答を生成するため、チャットテンプレートを適用し、言語モデルに与えるべきプロンプトを作成せよ。また、そのプロンプトに対する応答を生成し、表示せよ。

In [ ]:
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

prompt = "User: What do you call a sweet eaten after dinner? AI: "
encoded_input = tokenizer(prompt, return_tensors='pt')
input_ids = encoded_input['input_ids']

with torch.no_grad():
    outputs = model.generate(
        input_ids,
        max_new_tokens=10,
        do_sample=True,
        top_k=50,
        top_p=1.0,
        temperature=1.0
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("=== 応答 ===")
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


=== 応答 ===
User: What do you call a sweet eaten after dinner? AI: ~~~The dessert will be made from the fruit


## 95. マルチターンのチャット

問題94で生成された応答に対して、追加で"Please give me the plural form of the word with its spelling in reverse order."と問いかけたときの応答を生成・表示せよ。また、その時に言語モデルに与えるプロンプトを確認せよ。

In [ ]:
second_prompt = response + " User: Please give me the plural form of the word with its spelling in reverse order. AI:"
print(second_prompt)

encoded_input = tokenizer(second_prompt, return_tensors='pt')
input_ids = encoded_input['input_ids']
with torch.no_grad():
    outputs = model.generate(
        input_ids,
        max_new_tokens=10,
        do_sample=True,
        top_k=50,
        top_p=1.0,
        temperature=1.0
    )

second_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("=== 応答 ===")
print(second_response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


User: What do you call a sweet eaten after dinner? AI: ~~~The dessert will be made from the fruit User: Please give me the plural form of the word with its spelling in reverse order. AI:
=== 応答 ===
User: What do you call a sweet eaten after dinner? AI: ~~~The dessert will be made from the fruit User: Please give me the plural form of the word with its spelling in reverse order. AI: ~~~ The sweet will be made into something


## 96. プロンプトによる感情分析

事前学習済み言語モデルで感情分析を行いたい。テキストを含むプロンプトを事前学習済み言語モデルに与え、（ファインチューニングは行わずに）テキストのポジネガを予測するという戦略で、[SST-2](https://dl.fbaipublicfiles.com/glue/data/SST-2.zip)の開発データにおける正解率を測定せよ。

In [ ]:
# データの読み込み
train_df = pd.read_csv("SST-2/train.tsv", sep='\t', )
dev_df = pd.read_csv("SST-2/dev.tsv", sep='\t', )

dev_data = list(zip(dev_df['sentence'], dev_df['label'])) #list of tuple


In [ ]:
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
def classify(text, model, tokenizer):
  prompt = f"Review: {text} Sentiment: "
  inputs = tokenizer(prompt, return_tensors='pt')

  with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

  next_token_logits = logits[0, -1, :] #次の単語のlogits

  positive_token_id = tokenizer.encode(' positive')[0]
  negative_token_id = tokenizer.encode(' negative')[0]

  positive_logit = next_token_logits[positive_token_id]
  negative_logit = next_token_logits[negative_token_id]

  return 1 if positive_logit > negative_logit else 0

In [ ]:
correct = 0

for tuple in tqdm(dev_data):
  text = tuple[0]
  label = tuple[1]
  pred = classify(text, model, tokenizer)
  if pred == label:
    correct += 1


100%|██████████| 872/872 [03:03<00:00,  4.74it/s]


In [ ]:
print(correct/ len(dev_data)) #これで本当にいいのか...

0.6605504587155964


## 97. 埋め込みに基づく感情分析

事前学習済み言語モデルでテキストをベクトルで表現（エンコード）し、そのベクトルにフィードフォワード層を通すことで極性ラベルを予測するモデルを学習せよ。

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = 'distilgpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))


Embedding(50257, 768)

In [ ]:
class SST2Dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=128):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sentence = self.data.iloc[index]['sentence']
        label = int(self.data.iloc[index]['label'])

        encoding = self.tokenizer(
            sentence,
            truncation=True,#max_lengthよりも長い場合に自動的にトリミングするようにしてる
            max_length=self.max_len,
            padding='max_length',
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(0), #__getitem__()は１つのサンプルを返す関数だからバッチ次元を取り除いた方がいいっぽい
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.long)
        }

# 3. Datasetの作成
train_dataset = SST2Dataset(train_df, tokenizer)
dev_dataset = SST2Dataset(dev_df, tokenizer)

In [ ]:
ex = train_dataset.__getitem__(0)
ex

{'input_ids': tensor([24717,   649,  3200,   507,   422,   262, 21694,  4991,   220, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=16, shuffle=False)

i = 0
for data in train_dataloader:
    print(data["input_ids"].shape) #shape: [batch_size, max_length]
    print(data['label'].shape) #shape:[batch_size]
    print(data)
    i = i+1
    if i > 0:
      break #確認してみた

torch.Size([16, 128])
torch.Size([16])
{'input_ids': tensor([[ 5562, 14509,   494,  ..., 50256, 50256, 50256],
        [ 9442,   287,  1242,  ..., 50256, 50256, 50256],
        [  271,  1364, 36907,  ..., 50256, 50256, 50256],
        ...,
        [ 8117,   705,    82,  ..., 50256, 50256, 50256],
        [   69, 15451,   220,  ..., 50256, 50256, 50256],
        [  505,   561,   307,  ..., 50256, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'label': tensor([0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1])}


In [ ]:
import torch
import torch.nn as nn

class SentimentClassifier(nn.Module):
    def __init__(self, pretrained_model, num_categories=2, mlp_hidden_size=256, dropout_rate=0.3, loss_function=None):
        super().__init__()
        self.pretrained_model = pretrained_model

        for param in self.pretrained_model.parameters():
            param.requires_grad = False

        self.classifier_head = nn.Sequential(
            nn.Linear(pretrained_model.config.hidden_size, mlp_hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(mlp_hidden_size, num_categories)
        )

        self.loss_function = loss_function if loss_function is not None else nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.pretrained_model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
        hidden_state = outputs.hidden_states[-1]  # (batch_size, seq_len, hidden_dim)

        # attention_maskが1の最後の位置を取得（バッチ対応）
        last_token_indices = attention_mask.sum(dim=1) - 1  # shape: (batch_size,)
        pooled_output = hidden_state[torch.arange(hidden_state.size(0)), last_token_indices]

        logits = self.classifier_head(pooled_output)

        loss = None
        if labels is not None:
            loss = self.loss_function(logits, labels)

        return loss, logits


In [ ]:
classifier_with_gpt2 = SentimentClassifier(model)
classifier_with_gpt2.to(device)

SentimentClassifier(
  (pretrained_model): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-5): 6 x GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D(nf=2304, nx=768)
            (c_proj): Conv1D(nf=768, nx=768)
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D(nf=3072, nx=768)
            (c_proj): Conv1D(nf=768, nx=3072)
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (lm_head): Linear(in_features=7

In [ ]:
optimizer = torch.optim.Adam(classifier_with_gpt2.parameters(), lr=1e-4)
for epoch in range(1):
    classifier_with_gpt2.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch in tqdm(train_dataloader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        loss, logits = classifier_with_gpt2(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(logits, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    acc = correct / total
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}, Accuracy: {acc:.4f}")

100%|██████████| 4210/4210 [07:35<00:00,  9.24it/s]

Epoch 1, Loss: 2356.2462, Accuracy: 0.7075


In [ ]:
from torch.utils.data import DataLoader
from tqdm import tqdm

classifier_with_gpt2.eval()
total = 0
correct = 0

with torch.no_grad():
    for batch in tqdm(dev_dataloader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        # forward実行
        _, logits = classifier_with_gpt2(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(logits, dim=1)

        correct += (preds == labels).sum().item()
        total += labels.size(0)

dev_acc = correct / total
print(f"Dev Accuracy: {dev_acc:.4f}")

100%|██████████| 55/55 [00:05<00:00,  9.53it/s]

Dev Accuracy: 0.7993


## 98. ファインチューニング

問題96のプロンプトに対して、正解の感情ラベルをテキストの応答として返すように事前学習済みモデルをファインチューニングせよ。

In [ ]:
class GPT2SST2Dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=128):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.positive_token_id = tokenizer.encode(' positive')[0]
        self.negative_token_id = tokenizer.encode(' negative')[0]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sentence = self.data.iloc[index]['sentence']
        label = int(self.data.iloc[index]['label'])

        encoding = self.tokenizer(
            sentence,
            truncation=True,
            max_length=self.max_len,
            padding='max_length',
            return_tensors='pt'
        )

        label_token_id = self.positive_token_id if label == 1 else self.negative_token_id

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label_token_id': torch.tensor(label_token_id, dtype=torch.long),
            'labels': torch.tensor(label, dtype=torch.long)
        }



In [ ]:
class SentimentClassifier2(nn.Module):
    def __init__(self, pretrained_model, loss_function=None):
        super().__init__()
        self.pretrained_model = pretrained_model
        self.loss_function = loss_function if loss_function is not None else nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.pretrained_model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits  # (batch_size, seq_len, vocab_size)

        lengths = attention_mask.sum(dim=1) - 1  # (batch_size,)
        batch_size = input_ids.size(0)

        logits_last = logits[torch.arange(batch_size), lengths]  # (batch_size, vocab_size)

        loss = None
        if labels is not None:
            loss = self.loss_function(logits_last, labels)

        return loss, logits_last


In [ ]:
train_df = pd.read_csv("SST-2/train.tsv", sep='\t', )
dev_df = pd.read_csv("SST-2/dev.tsv", sep='\t', )
train_df['sentence'] = train_df['sentence'].apply(lambda x: f"Review: {x} Sentiment: ")
dev_df['sentence'] = dev_df['sentence'].apply(lambda x: f"Review: {x} Sentiment: ") #プロンプトの加工

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = 'distilgpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model_dx = GPT2LMHeadModel.from_pretrained(model_name)
model_dx.resize_token_embeddings(len(tokenizer))
model_dx.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
# ダミー入力
sentence = "This is a test."
inputs = tokenizer(sentence, return_tensors='pt').to(device)

# モデルを通す
outputs = model_dx(**inputs)
logits = outputs.logits

outputs.logits.shape

torch.Size([1, 5, 50257])

In [ ]:
train_dataset = GPT2SST2Dataset(train_df, tokenizer)
dev_dataset = GPT2SST2Dataset(dev_df, tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=16, shuffle=False)

In [ ]:
ex = train_dataset.__getitem__(0)
ex

{'input_ids': tensor([14832,    25,  7808,   649,  3200,   507,   422,   262, 21694,  4991,
           220, 11352,  3681,    25,   220, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50

In [ ]:
classifier_with_gpt2_dx = SentimentClassifier2(model_dx)
classifier_with_gpt2_dx.to(device)

SentimentClassifier2(
  (pretrained_model): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-5): 6 x GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D(nf=2304, nx=768)
            (c_proj): Conv1D(nf=768, nx=768)
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D(nf=3072, nx=768)
            (c_proj): Conv1D(nf=768, nx=3072)
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (lm_head): Linear(in_features=

In [ ]:
positive_id = tokenizer.convert_tokens_to_ids("positive")
negative_id = tokenizer.convert_tokens_to_ids("negative")
print(positive_id)
print(negative_id)

24561
31591


In [ ]:
optimizer = torch.optim.Adam(classifier_with_gpt2_dx.parameters(), lr=1e-4)
for epoch in range(1):
    classifier_with_gpt2_dx.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch in tqdm(train_dataloader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        label_id = batch["label_token_id"].to(device)
        label = batch["labels"].to(device)


        optimizer.zero_grad()
        loss, logits = classifier_with_gpt2_dx(input_ids=input_ids, attention_mask=attention_mask, labels=label_id)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        logits_selected = logits[:, [negative_id, positive_id]]
        preds = torch.argmax(logits_selected, dim=1)  # 0: negative, 1: positive
        correct += (preds == label).sum().item()
        total += label.size(0)

    acc = correct / total
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}, Accuracy: {acc:.4f}")

100%|██████████| 4210/4210 [21:37<00:00,  3.24it/s]

Epoch 1, Loss: 976.3962, Accuracy: 0.9077


In [ ]:
from torch.utils.data import DataLoader
from tqdm import tqdm

classifier_with_gpt2_dx.eval()
total = 0
correct = 0

with torch.no_grad():
    for batch in tqdm(dev_dataloader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        label_id = batch["label_token_id"].to(device)
        label = batch["labels"].to(device)

        # forward実行
        _, logits = classifier_with_gpt2_dx(input_ids=input_ids, attention_mask=attention_mask)
        logits_selected = logits[:, [negative_id, positive_id]]
        preds = torch.argmax(logits_selected, dim=1)  # 0: negative, 1: positive
        correct += (preds == label).sum().item()
        total += label.size(0)

dev_acc = correct / total
print(f"Dev Accuracy: {dev_acc:.4f}")

100%|██████████| 55/55 [00:06<00:00,  8.22it/s]

Dev Accuracy: 0.8933


## 99. 選好チューニング

問題96のプロンプトに対して、正解の感情ラベルを含むテキストを望ましい応答、間違った感情ラベルを含むテキストを望ましくない応答として、事前学習済み言語モデルを選好チューニング (preference tuning) を実施せよ。選好チューニングのアルゴリズムとしては、近傍方策最適化 (PPO: Proximal Policy Optimization) や直接選好最適化 (DPO: Direct Preference Optimization) などが考えられる。


In [1]:
%%capture
!pip install transformers, datasets
!pip install trl

In [2]:
%%capture
!wget https://dl.fbaipublicfiles.com/glue/data/SST-2.zip
!unzip SST-2.zip

In [6]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments
import pandas as pd
from datasets import Dataset
from trl import DPOTrainer #99.DPOに使用
import torch

In [4]:
# データの読み込み
train_df = pd.read_csv("SST-2/train.tsv", sep='\t', )
dev_df = pd.read_csv("SST-2/dev.tsv", sep='\t', )

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = 'distilgpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model_dx = GPT2LMHeadModel.from_pretrained(model_name)
model_dx.config.pad_token_id = tokenizer.pad_token_id
model_dx.resize_token_embeddings(len(tokenizer))
model_dx.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [8]:
#https://github.com/upura/nlp100v2025/blob/main/ch10/ans99.py の関数を一部改変し使用


def create_prompt(sentence):
    return f"""Analyze the sentiment of the following sentence. Answer with either "positive" or "negative" after the sentence.

sentence: {sentence}
sentiment: """


def create_preference_data(df):
    # 正解ラベルに基づいて望ましい応答と望ましくない応答を作成
    chosen_responses = []
    rejected_responses = []
    prompts = []

    for _, row in df.iterrows():
        sentence = row["sentence"]
        label = row["label"]

        # プロンプトの作成
        prompt = create_prompt(sentence)
        prompts.append(prompt)

        # 正解ラベルに基づいて応答を作成
        if label == 1:  # positive
            chosen_responses.append(prompt + "positive")
            rejected_responses.append(prompt + "negative")
        else:  # negative
            chosen_responses.append(prompt + "negative")
            rejected_responses.append(prompt + "positive")

    return {
        "prompt": prompts,
        "chosen": chosen_responses,
        "rejected": rejected_responses,
    }

In [9]:
train_preference_data = create_preference_data(train_df)
dev_preference_data = create_preference_data(dev_df)

train_dataset = Dataset.from_dict(train_preference_data)
dev_dataset = Dataset.from_dict(dev_preference_data)

In [11]:
from trl import DPOTrainer, DPOConfig

# TrainingArguments の代わりに DPOConfig を使う
training_args = DPOConfig(
    output_dir="./results_dpo",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    warmup_ratio=0.1,
    num_train_epochs=3,
    logging_steps=1,
    optim="adamw_8bit",
    seed=42,
)

# DPOトレーナーの作成
dpo_trainer = DPOTrainer(
    model=model_dx,
    ref_model=None,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
)

# 学習の実行
dpo_trainer.train()

# 最終評価
eval_results = dpo_trainer.evaluate()
print(f"最終評価結果: {eval_results}")



ValueError: processing_class must be specified to tokenize a DPO dataset.